### Imports

In [2]:
import os
import re
from typing import Dict

import mne
import numpy as np
import scipy.stats as sp_stats
from joblib import Parallel, delayed
from NirsLabProject.config.paths import Paths

from NirsLabProject.utils import pipeline_utils as putils
from NirsLabProject.config.consts import *

from NirsLabProject.utils import general_utils as utils
from NirsLabProject.config.consts import *
from NirsLabProject.config.subject import Subject
from NirsLabProject.utils.group_spikes import group_spikes
from NirsLabProject.utils import plotting


### Extract Paths of stimuli features
These are the features from the raw edf data without grouping, stimuli and sleeping features

In [3]:
subjects = [Subject(d, True) for d in os.listdir(Paths.products_data_dir_path) if d.startswith('p')]
stimuli_subjects = subjects
stimuli_subjects_with_features = filter(lambda subj: os.path.exists(subj.paths.subject_channels_spikes_features_path), stimuli_subjects)
stimuli_subjects_with_features = sorted(stimuli_subjects_with_features, key=lambda s: s.p_number)

In [4]:
def edit_coordinates(channels_spikes_features):
    channel_name_to_coordinates = utils.calculate_coordinates(subj)
    for channel, data in channels_spikes_features.items():
        x, y, z = utils.get_coordinates_of_channel(channel_name_to_coordinates, channel)
        data[:, CORD_X_INDEX] = x
        data[:, CORD_Y_INDEX] = y
        data[:, CORD_Z_INDEX] = z
    return channels_spikes_features

def regenerate_flat_features(subj: Subject):
    channels_spikes_features = np.load(
        subj.paths.subject_channels_spikes_features_path,
        allow_pickle=True
    ).item()
    channels_spikes_features = edit_coordinates(channels_spikes_features)
    index_to_channel = putils.get_index_to_channel(subj, channels_spikes_features)
    groups, flat_features, all_spikes_group_focal_point_amplitude, _ = putils.group_spikes(subj, channels_spikes_features, index_to_channel)
    flat_features = utils.add_flag_of_scalp_detection_to_spikes_features(flat_features, None)
    flat_features = utils.add_stimuli_flag_to_spikes_features(subj, flat_features)
    flat_features = utils.add_sleeping_stage_flag_to_spike_features(subj, flat_features)
    subject_id = np.ones((flat_features.shape[0], 1)) * subj.p_number
    flat_features = np.concatenate((flat_features, subject_id, all_spikes_group_focal_point_amplitude.reshape((-1, 1))), axis=1)
    return flat_features

for subj in stimuli_subjects_with_features:
    flat_features = regenerate_flat_features(subj)
    np.save(subj.paths.subject_flat_features_path, flat_features)
    

calculating coordinates for subject 13
channel RMTG5 not found in coordinates file
channel RMTG7 not found in coordinates file
Grouping spikes
Could not get sleeping stages for subject 13, error: No hypnogram file found for subject p13
calculating coordinates for subject 17
channel LAI6 not found in coordinates file
channel LAI7 not found in coordinates file
channel LMI7 not found in coordinates file
channel LPI7 not found in coordinates file
channel LSI7 not found in coordinates file
channel LSTG5 not found in coordinates file
channel LSTG6 not found in coordinates file
channel LSTG7 not found in coordinates file
Grouping spikes
Could not get sleeping stages for subject 17, error: No hypnogram file found for subject p17
calculating coordinates for subject 18
channel LOF-AC1 not found in coordinates file
channel LOF-AC2 not found in coordinates file
channel LOF-AC3 not found in coordinates file
channel LOF-AC5 not found in coordinates file
channel LOF-AC6 not found in coordinates file


Could not get sleeping stages for subject 46, error: No hypnogram file found for subject p46
calculating coordinates for subject 51
Grouping spikes
Could not get sleeping stages for subject 51, error: No hypnogram file found for subject p51
calculating coordinates for subject 396
no coordinates file found, calculating coordinates from average coordinates files
Grouping spikes
calculating coordinates for subject 398
no coordinates file found, calculating coordinates from average coordinates files
channel RACOF1 not found in coordinates file
channel RACOF2 not found in coordinates file
channel RACOF3 not found in coordinates file
channel RACOF4 not found in coordinates file
channel RACOF5 not found in coordinates file
channel RACOF6 not found in coordinates file
channel RTO1 not found in coordinates file
channel RTO2 not found in coordinates file
channel RTO3 not found in coordinates file
channel RTO4 not found in coordinates file
channel RTO5 not found in coordinates file
channel RTO6 n

In [9]:
import shutil
import os

for subj in stimuli_subjects_with_features:
    new_path = '/tmp/flat_features'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    shutil.copyfile(subj.paths.subject_flat_features_path, os.path.join(new_path, f'{subj.p_number}_flat_features.npy'))